### Dependencias

In [1]:
#Librerias
import pandas as pd
pd.set_option('display.float_format', '{:,.2f}'.format)
import numpy as np
import os
import pyodbc
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from sqlalchemy import create_engine
from datetime import datetime
import re
from thefuzz import process
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from PyPDF2 import PdfMerger
import json
from snowflake.connector import ProgrammingError
# Cargar variables desde .env
load_dotenv()

True

In [2]:
paises_snowflake = {
    'brasil': ("Brasil", (22,)),
    'caricam': ("CARICAM", (14, 15, 16, 17, 19, 20)),
    'eu': ("EU", (8,)),
    'paraguay': ("Paraguay", (24,)),
    'colombia': ("Colombia", (6,)),
    'ecuador': ("Ecuador", (12,)),
    'uruguay': ("Uruguay", (23,)),
    'chile': ("Chile", (13,)),
    'dominicana': ("Rep. Dominicana", (21,)),
    'peru': ("Perú", (10,))
}
columnas_sumar = [
    'Precio Unitario VTA GLI', 'Unidades Desplazadas',
    'Unidades Inv. CEDIS', 'Unidades Inv. Transito', 'Unidades Inv. Tienda',
    'Monto Desplazado Bruto ML'
]

### 👉 Aquí modificar el país

In [3]:
# MODIFICAR
PaisCarga ="peru"  # Cambiar por el país deseado
# No es necesario modificar
PaisCarpeta = paises_snowflake[PaisCarga.lower()][0]
PaisSF = paises_snowflake[PaisCarga.lower()][1]

### Funciones

In [4]:

def extraer_rango_semanas(nombre_archivo):
    """
    Devuelve (inicio, fin) como enteros YYYYWW.
    Reconoce: YYYYWW, YYYY WW, WW YYYY y WWYYYY (p.ej., 312025).
    """
    s = re.sub(r'\.[^.]+$', '', str(nombre_archivo))
    pares = []

    def ok_week(w): return 1 <= w <= 53
    def add(y, w):
        if 2000 <= y <= 2099 and ok_week(w):
            pares.append((y, w))

    # Formatos separados
    for y, w in re.findall(r'(20\d{2})\D+([1-9]|[1-4]\d|5[0-3])', s):
        add(int(y), int(w))
    for w, y in re.findall(r'([1-9]|[1-4]\d|5[0-3])\D+(20\d{2})', s):
        add(int(y), int(w))

    # Tokens compactos de 6 dígitos
    for t in re.findall(r'(?<!\d)(\d{6})(?!\d)', s):
        y, w = int(t[:4]), int(t[4:6])        # YYYYWW
        if 2000 <= y <= 2099 and ok_week(w):
            add(y, w)
            continue
        w, y = int(t[:2]), int(t[2:])         # WWYYYY
        if ok_week(w) and 2000 <= y <= 2099:
            add(y, w)

    if len(pares) < 2:
        raise ValueError("No se encontraron al menos dos (AÑO,SEMANA) válidos.")

    (y1, w1), (y2, w2) = pares[-2], pares[-1]
    return y1 * 100 + w1, y2 * 100 + w2



In [5]:
def resumir_dataframe(df, columnas_sumar,desc,montos_faltan=None):
 
    # Convertir columnas de suma a float si existen y no lo son

    columnas_sumar_existentes = [col for col in columnas_sumar if col in df.columns]
    for col in columnas_sumar_existentes:
        if col in df.columns and not pd.api.types.is_float_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], errors='coerce')
    # Calcular resumen)
    suma_totales = df[columnas_sumar_existentes].sum()
    if montos_faltan is not None and not montos_faltan.empty:
        suma_totales = suma_totales.add(montos_faltan, fill_value=0)

    # Unir ambos resultados
    resumen = suma_totales.reset_index()
    resumen.columns = ['Variable', f'Resumen_{desc}']
    # Agregar columna con longitud del DataFrame original
    resumen.loc[len(resumen)] = ['Total Filas en df', len(df)]
    return resumen

In [6]:
def exportar_rechazos_por_campo(
    df: pd.DataFrame,
    campo_faltante: str,
    columnas_agrupacion: list,
    columnas_suma: list,
    carpeta_pais: str,
    pais: str
):
    try:
        # Filtrar rechazos
        df_rechazos = df[df[campo_faltante].isna()]
        df_rechazos = df_rechazos.dropna(axis=1, how='all')
        if df_rechazos.empty:
            print(f"Sin rechazos de {campo_faltante}")
            return None
        # Agrupar datos con las que columnas que sí existan en el DataFrame
        columnas_agrupacion = [col for col in columnas_agrupacion if col in df_rechazos.columns]
        columnas_suma = [col for col in columnas_suma if col in df_rechazos.columns]
        df_rechazos_agrupado = df_rechazos.groupby(columnas_agrupacion, as_index=False)[columnas_suma].sum()
        # Eliminar registros donde todas las columnas de suma sean cero
        #df_rechazos_agrupado = df_rechazos_agrupado.loc[df_rechazos_agrupado[columnas_suma].sum(axis=1) != 0]
        # Agregar columnas adicionales
        df_rechazos_agrupado['Fecha Rechazo'] = datetime.now().strftime("%Y-%m-%d")
        #df_rechazos_agrupado['Sem Métrica'] = semana
        df_rechazos_agrupado['Métrica'] = 'SO'
        df_rechazos_agrupado.rename(columns={'PAISID': 'País', 'Nombre Grupo':'GrpNombre', 'Nombre Cadena/Formato':'Cadena', 
                                            'GRPID':'GrpID', 'Descripción presentación de producto':'Descripción','Material SAP':'Código (SAP/ERP)', 
                                            'Cód. Interno ERP':'Código (SAP/ERP)'}, inplace=True)
        df_rechazos_agrupado.drop(columns=['SEMID'], inplace=True, errors='ignore')
        
        # Crear subcarpeta "Rechazos" si no existe
        carpeta_base = r'C:\Users\elsilva\OneDrive - genommalabinternacional\Layout Internacional\data\Rechazos' 
        carpeta_rechazos = os.path.join(carpeta_base, pais)
        # Crear carpeta si no existe
        os.makedirs(carpeta_rechazos, exist_ok=True)
        # Exportar archivo
        hoy = datetime.now().strftime("%Y%m%d")
        salida = os.path.join(carpeta_rechazos, f"Rechazos_{campo_faltante}_{pais}_{hoy}.xlsx")
        df_rechazos_agrupado.to_excel(salida, index=False)
        print(f"Se generaron rechazos de {campo_faltante}: {salida}")
        return df_rechazos_agrupado
    except Exception as e:
        print(f"Error al exportar rechazos de {campo_faltante}: {e}")
        return None

In [7]:
def read_sql_upper(query: str, engine, params=None):
    try:
        df = pd.read_sql(query, engine, params=params)
        if df.empty:
            print(f"El DataFrame resultante está vacío de {query}.")
            return pd.DataFrame(columns=[col.upper() for col in df.columns])
        df.columns = [col.upper() for col in df.columns]
        return df
    except Exception as e:
        raise RuntimeError(f"Error al ejecutar la consulta SQL o procesar los resultados: {e}")

In [8]:
def fuzzy_merge(df_base, df_catalogo, campo_base, campo_catalogo, campos_join_adicionales=[], campo_valor='GRPID', score_minimo=80):
    df = df_base.copy()
    # Paso 1: merge exacto
    df_merged = df.merge(
        df_catalogo[campos_join_adicionales + [campo_catalogo, campo_valor]].drop_duplicates(),
        left_on=campos_join_adicionales + [campo_base],
        right_on=campos_join_adicionales + [campo_catalogo],
        how='left'
    )
    # Paso 2: filtrar los que no hicieron match exacto
    no_match = df_merged[df_merged[campo_valor].isna()].copy()
    no_match_unique = no_match[campos_join_adicionales + [campo_base]].drop_duplicates()
    # Paso 3: aplicar fuzzy matching por grupo (por ejemplo, PAISID)
    matches = []
    for keys, grupo in no_match_unique.groupby(campos_join_adicionales):
        base_valores = grupo[campo_base].dropna().unique()
        # Filtra catálogo por valores adicionales
        mask = (df_catalogo[campos_join_adicionales] == pd.Series(keys, index=campos_join_adicionales)).all(axis=1)
        catalogo_valores = df_catalogo[mask][campo_catalogo].dropna().unique()
        for valor in base_valores:
            # Calculan las operaciones mínimas necesarias para convertir una caden en otra
            match, score = process.extractOne(valor, catalogo_valores)
            if score >= score_minimo:
                matches.append({
                    **dict(zip(campos_join_adicionales, keys)),
                    campo_base: valor,
                    f'{campo_catalogo}_MATCH': match,
                    'SCORE_MATCH': score
                })

    # Paso 4: unir fuzzy matches y traer valor del catálogo
    df_match = pd.DataFrame(matches)
    if df_match.empty:
        return df_merged, df_match # nada que actualizar
    df_match = df_match.merge(
        df_catalogo[campos_join_adicionales + [campo_catalogo, campo_valor]].drop_duplicates(),
        left_on=campos_join_adicionales + [f'{campo_catalogo}_MATCH'],
        right_on=campos_join_adicionales + [campo_catalogo],
        how='left'
    )
    # Paso 5: actualizar solo los valores faltantes (sin MultiIndex)
    for col in [campo_valor, f'{campo_catalogo}_MATCH', 'SCORE_MATCH']:
        if col in df_match.columns:
            cols_para_merge = campos_join_adicionales + [campo_base, col]
            df_merged = df_merged.merge(
                df_match[cols_para_merge],
                on=campos_join_adicionales + [campo_base],
                how='left',
                suffixes=('', '_FUZZY')
            )
            col_fuzzy = f"{col}_FUZZY"
            if col_fuzzy in df_merged.columns:
                df_merged[col] = df_merged[col].combine_first(df_merged[col_fuzzy])
                df_merged.drop(columns=[col_fuzzy], inplace=True)
    return df_merged.reset_index(), df_match


In [9]:
def resaltar_diferencias(row, columnas, df_diferencias, color='#ffb3c6'):
    estilos = [''] * len(row)
    for i, col in enumerate(row.index):
        if col.endswith('_insertado'):
            base_col = col.replace('_insertado', '')
            if base_col in df_diferencias.columns and df_diferencias.loc[row.name, base_col]:
                estilo = f'background-color: {color}; color: #333;'
                estilos[i] = estilo
                idx_base = list(row.index).index(base_col)
                estilos[idx_base] = estilo
    return estilos


In [10]:
def graficar_por_tipocomprod_interactivo(df_hist, df_nuevo_con_rechazos, campos, df_nuevo_sin_rechazos=None, campo_x='SEMID'): 
    """
    Grafica datos históricos, nuevos y opcionalmente nuevos sin rechazos por tipo de producto.
    
    Args:
        df_hist: DataFrame con datos históricos
        df_nuevo: DataFrame con datos nuevos
        campos: Lista de campos a graficar
        df_nuevo_sin_rechazos: DataFrame opcional con datos nuevos sin rechazos
        campo_x: Campo para el eje X (default: 'SEMID')
    """
    # Obtener todos los tipos únicos de productos de todos los DataFrames disponibles
    tipos_set = set(df_hist['PAISID']).union(df_nuevo_con_rechazos['PAISID'])
    
    if df_nuevo_sin_rechazos is not None:
        tipos_set = tipos_set.union(df_nuevo_sin_rechazos['PAISID'])
    
    tipos = sorted(tipos_set)
    
    # Generar un gráfico por cada tipo de producto
    for tipo in tipos:
        # Filtrar datos para el tipo actual
        df_hist_tipo = df_hist[df_hist['PAISID'] == tipo]
        df_nuevo_tipo = df_nuevo_con_rechazos[df_nuevo_con_rechazos['PAISID'] == tipo]
        
        # Filtrar datos nuevos sin rechazos si está disponible
        df_nuevo_sin_rechazos_tipo = None
        if df_nuevo_sin_rechazos is not None:
            df_nuevo_sin_rechazos_tipo = df_nuevo_sin_rechazos[df_nuevo_sin_rechazos['PAISID'] == tipo]

        # Omitir si todos los DataFrames están vacíos para este tipo
        if (df_hist_tipo.empty and 
            df_nuevo_tipo.empty and 
            (df_nuevo_sin_rechazos_tipo is None or df_nuevo_sin_rechazos_tipo.empty)):
            continue

        # Obtener país (priorizar datos históricos, luego nuevos, luego sin rechazos)
        pais = "Desconocido"
        if not df_hist_tipo.empty and 'PAIS' in df_hist_tipo.columns:
            paises = df_hist_tipo['PAIS'].unique()
            pais = paises[0] if len(paises) > 0 else "Desconocido"
        elif not df_nuevo_tipo.empty and 'PAIS' in df_nuevo_tipo.columns:
            paises = df_nuevo_tipo['PAIS'].unique()
            pais = paises[0] if len(paises) > 0 else "Desconocido"
        elif (df_nuevo_sin_rechazos_tipo is not None and 
              not df_nuevo_sin_rechazos_tipo.empty and 
              'PAIS' in df_nuevo_sin_rechazos_tipo.columns):
            paises = df_nuevo_sin_rechazos_tipo['PAIS'].unique()
            pais = paises[0] if len(paises) > 0 else "Desconocido"

        # Crear figura con subgráficos (1 fila, n columnas)
        n = len(campos)
        fig = make_subplots(
            rows=1, 
            cols=n, 
            subplot_titles=campos,
            shared_xaxes=True,
            shared_yaxes=False
        )

        fig.update_layout(
            title=f'Comparación por país: {pais}, tipo: {tipo}',
            showlegend=True,
            height=400,
            width=400 * n
        )

        # Añadir gráficos por campo
        for i, campo in enumerate(campos, 1):
            # Datos históricos
            if not df_hist_tipo.empty and campo in df_hist_tipo.columns:
                fig.add_trace(
                    go.Scatter(
                        x=df_hist_tipo[campo_x],
                        y=df_hist_tipo[campo],
                        name='Histórico',
                        mode='lines+markers',
                        marker=dict(color='blue'),
                        showlegend=(i == 1)
                    ),
                    row=1, col=i
                )

            # Datos nuevos (con rechazos)
            if not df_nuevo_tipo.empty and campo in df_nuevo_tipo.columns:
                if len(df_nuevo_tipo) == 1:
                    fig.add_trace(
                        go.Scatter(
                            x=df_nuevo_tipo[campo_x],
                            y=df_nuevo_tipo[campo],
                            name='Nuevo con rechazos',
                            mode='markers',
                            marker=dict(color='red', size=10),
                            showlegend=(i == 1)
                        ),
                        row=1, col=i
                    )
                else:
                    fig.add_trace(
                        go.Scatter(
                            x=df_nuevo_tipo[campo_x],
                            y=df_nuevo_tipo[campo],
                            name='Nuevo con rechazos',
                            mode='lines+markers',
                            line=dict(dash='dash'),
                            marker=dict(color='red', symbol='x'),
                            showlegend=(i == 1)
                        ),
                        row=1, col=i
                    )

            # Datos nuevos sin rechazos (si está disponible)
            if (df_nuevo_sin_rechazos_tipo is not None and 
                not df_nuevo_sin_rechazos_tipo.empty and 
                campo in df_nuevo_sin_rechazos_tipo.columns):
                
                if len(df_nuevo_sin_rechazos_tipo) == 1:
                    fig.add_trace(
                        go.Scatter(
                            x=df_nuevo_sin_rechazos_tipo[campo_x],
                            y=df_nuevo_sin_rechazos_tipo[campo],
                            name='Nuevo sin rechazos',
                            mode='markers',
                            marker=dict(color='green', size=10, symbol='diamond'),
                            showlegend=(i == 1)
                        ),
                        row=1, col=i
                    )
                else:
                    fig.add_trace(
                        go.Scatter(
                            x=df_nuevo_sin_rechazos_tipo[campo_x],
                            y=df_nuevo_sin_rechazos_tipo[campo],
                            name='Nuevo sin rechazos',
                            mode='lines+markers',
                            line=dict(dash='dot'),
                            marker=dict(color='green', symbol='diamond'),
                            showlegend=(i == 1)
                        ),
                        row=1, col=i
                    )

            # Ejes
            fig.update_xaxes(title_text=campo_x, row=1, col=i)
            if i == 1:
                fig.update_yaxes(title_text='Valor', row=1, col=i)

        # Mostrar gráfico
        fig.show()

In [11]:
def generar_grafico_comparativo(
    df_hec, 
    df_dm, 
    campo_metrica="SOUTMONTODESPBRT",
    titulo="Comparativo HEC vs DM con Banda de Desviación",
    archivo_salida="pagina_figura.pdf",
    mostrar_bandas=True,
    referencia_estadisticas="dm"
):
    """
    Genera gráfico comparativo entre HEC y DM con bandas de desviación
    
    Parámetros:
    -----------
    df_hec : DataFrame
        DataFrame con datos de HEC
    df_dm : DataFrame
        DataFrame con datos de DM
    campo_metrica : str, default "soutmontodesp"
        Campo métrica a comparar
    titulo : str
        Título del gráfico
    archivo_salida : str
        Nombre del archivo PDF de salida
    mostrar_bandas : bool, default True
        Si mostrar las bandas estadísticas
    referencia_estadisticas : str, default "dm"
        Sobre qué datos calcular estadísticas ("dm", "hec", o "ambos")
    
    Retorna:
    --------
    plotly.graph_objects.Figure
    """
    
    # Agrupar datos
    ventas_hec = (df_hec.groupby(['SEMANIO', 'SEMNUMERO'])[campo_metrica]
                  .sum()
                  .reset_index(name='ventas_hec'))
    
    ventas_dm = (df_dm.groupby(['SEMANIO', 'SEMNUMERO'])[campo_metrica]
                  .sum()
                  .reset_index(name='ventas_dm'))
    
    # Unir y procesar datos
    df_comparado = pd.merge(ventas_hec, ventas_dm, 
                           on=['SEMANIO', 'SEMNUMERO'], 
                           how='outer').fillna(0)
    
    df_comparado['semana_str'] = (df_comparado['SEMANIO'].astype(str) + 
                                 '-S' + df_comparado['SEMNUMERO'].astype(str))
    
    df_comparado = df_comparado.sort_values(['SEMANIO', 'SEMNUMERO'])
    df_comparado['diferencia'] = df_comparado['ventas_hec'] - df_comparado['ventas_dm']
    df_comparado['diferencia_abs'] = df_comparado['diferencia'].abs()
    
    # Crear figura
    fig = go.Figure()
    
    # Banda de desviación (para resaltar diferencia entre ambas líneas)
    fig.add_trace(go.Scatter(
        x=pd.concat([df_comparado['semana_str'], df_comparado['semana_str'][::-1]]),
        y=pd.concat([df_comparado['ventas_hec'], df_comparado['ventas_dm'][::-1]]),
        fill='toself',
        fillcolor='rgba(255, 0, 0, 0.25)',
        line=dict(color='rgba(255,255,255,0)'),
        name='Desviación',
        showlegend=True
    ))
    
    # Línea DM
    fig.add_trace(go.Scatter(
        x=df_comparado['semana_str'],
        y=df_comparado['ventas_dm'],
        mode='lines+markers',
        name='DM',
        line=dict(color='blue')
    ))
    
    # Línea HEC
    fig.add_trace(go.Scatter(
        x=df_comparado['semana_str'],
        y=df_comparado['ventas_hec'],
        mode='lines+markers',
        name='HEC',
        line=dict(color='green')
    ))
    
    # Configurar layout básico
    fig.update_layout(
        title=titulo,
        xaxis_title='Semana',
        yaxis_title=f'{campo_metrica.upper()}',
        legend_title='Fuente',
        hovermode='x unified'
    )
    
    fig.update_xaxes(tickfont=dict(size=9))
    
    # Agregar bandas estadísticas si se requiere
    if mostrar_bandas and len(df_comparado) > 1:
        # Determinar sobre qué datos calcular estadísticas
        if referencia_estadisticas.lower() == "dm":
            datos_ref = df_comparado['ventas_dm']
        elif referencia_estadisticas.lower() == "hec":
            datos_ref = df_comparado['ventas_hec']
        else:  # "ambos"
            datos_ref = pd.concat([df_comparado['ventas_dm'], df_comparado['ventas_hec']])
        
        media = datos_ref.mean()
        std = datos_ref.std()
        upper_band = media + std
        lower_band = media - std
        
        # Líneas estadísticas
        fig.add_hline(
            y=media, 
            line_dash="dash", 
            line_color="orange", 
            annotation_text="Media", 
            annotation_position="top left"
        )
        fig.add_hline(
            y=upper_band, 
            line_dash="dot", 
            line_color="red", 
            annotation_text="Media + STD", 
            annotation_position="top left"
        )
        fig.add_hline(
            y=lower_band, 
            line_dash="dot", 
            line_color="red", 
            annotation_text="Media - STD", 
            annotation_position="bottom left"
        )
    
    return fig


### Cargar archivo

In [12]:
ruta_base = r"C:\Users\elsilva\OneDrive - genommalabinternacional\Layout Internacional\data"
ruta_destino = r"C:\Users\elsilva\OneDrive - genommalabinternacional\Layout Internacional\Python\CargaSem"

carpeta_pais = os.path.join(ruta_base, PaisCarpeta)
carpeta_pais_destino = os.path.join(ruta_destino, PaisCarpeta)

if os.path.exists(carpeta_pais) and os.path.isdir(carpeta_pais):
        archivos_xlsx = [
            os.path.join(carpeta_pais, f)
            for f in os.listdir(carpeta_pais)
            if f.endswith(".xlsx") and os.path.isfile(os.path.join(carpeta_pais, f))
        ]
        if not archivos_xlsx:
            raise RuntimeError("No se encontró ningún archivo .xlsx en la carpeta.")
        # Ordenar por fecha de modificación descendente (más reciente primero)
        archivos_xlsx.sort(key=os.path.getmtime, reverse=True)
        ruta_archivo = archivos_xlsx[0]
        print(f"Archivo más reciente: {ruta_archivo}")
        try:
            df = pd.read_excel(
                ruta_archivo,sheet_name="LayOut_SellOut", skiprows=6,usecols="I:Y",engine='openpyxl',
                dtype={ #"Monto Desplazado Bruto ML": float,"Monto Desplazado Neto ML": float,"Factor Bruto Neto SellOut": float, 
                    'Nombre Cadena/Formato': str,'Nombre Grupo': str,}
            )
    
            for col in ["Monto Desplazado Bruto ML", "Monto Desplazado Neto ML", "Factor Bruto Neto SellOut", "Unidades Inv. Tienda"]:
                if col in df.columns:
                    df[col] = (
                        df[col]
                        .astype(str).str.strip()
                        .replace(r'[^0-9\.-]', '', regex=True)
                        .replace(r'^-$', '0', regex=True)
                        .replace(r'^$', '0', regex=True)
                        .astype(float)
                    )

            # Extraer el rango de semanas del nombre del archivo
            rango_semanas = extraer_rango_semanas(os.path.basename(ruta_archivo))
            semana_inicio, semana_fin = sorted(rango_semanas)
            print(F'Semanas a procesar : {semana_inicio}, {semana_fin}')
        except Exception as e:
            raise RuntimeError(f"Error al leer el archivo: {e}")
else:
    raise RuntimeError(f"La carpeta del país '{carpeta_pais}' no existe.")

Archivo más reciente: C:\Users\elsilva\OneDrive - genommalabinternacional\Layout Internacional\data\Perú\Perú_202534_202534.xlsx
Semanas a procesar : 202534, 202534


### Consultas

In [13]:
# Acceder a las variables
user = os.getenv("USER")
password = os.getenv("PASSWORD")
account = os.getenv("ACCOUNT")
role = os.getenv("ROLE")
# Configura el motor de conexión
conn_sf = create_engine(
    f'snowflake://{user}:{password}@{account}/?role={role}'
)
conexion = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    role = role
)

In [14]:
# Generar placeholders para cada país
placeholders_pais = ", ".join(["%s"] * len(PaisSF))
#Consulta Snowflake Tiempo
Query_catsemanas = f"""
  SELECT *
  FROM PRD_STG.GNM_CT.CATSEMANAS
  WHERE (SEMANIO * 100 + SEMNUMERO) BETWEEN {semana_inicio} AND {semana_fin};
  """
df_CatSem = read_sql_upper(Query_catsemanas, conn_sf)
#Consulta Snowflake TipoComProd
Query_Pais =  """
  SELECT PAIS, PAISID FROM
  PRD_CNS_MX.CATALOGOS.VW_DIM_PAIS
  """
df_Pais = read_sql_upper(Query_Pais, conn_sf)
df_Pais['PAIS'] = df_Pais['PAIS'].str.strip().str.upper()
#Consulta Snowflake Clientes
Query_Cliente = f"""
  SELECT PAISID, GRPID, GRPNOMBRE, CADID,CADNOMBRE
  FROM PRD_CNS_MX.CATALOGOS.VW_ESTRUCTURACLIENTESSEGPTVTOTAL
  WHERE  PAISID IN ({placeholders_pais})
  """
df_Cliente = read_sql_upper(Query_Cliente, conn_sf, PaisSF)
df_Cliente['CADNOMBRE'] = df_Cliente['CADNOMBRE'].str.strip().str.upper()
df_Cliente['GRPNOMBRE'] = df_Cliente['GRPNOMBRE'].str.strip().str.upper()
#Consulta Snowflake Productos
Query_Productos = f"""
  SELECT PAISID, PROID, PROPSTID, PROPSTCODBARRAS
  FROM PRD_CNS_MX.CATALOGOS.VW_ESTRUCTURAPRODUCTOSTOTALPAISES
  WHERE PAISID IN ({placeholders_pais})
  ORDER BY PROPSTID ASC
  """
df_Prod = read_sql_upper(Query_Productos, conn_sf, PaisSF)
df_Prod = df_Prod.drop_duplicates(subset=['PROPSTCODBARRAS','PAISID'], keep='first')
# Datos previos a la semana
Query_hist = f"""
  SELECT
    ft.PAISID, UPPER(p.PAIS) as PAIS,ft.SEMID, cs.SEMNUMERO, SUM(ft.SOUTCANTDESP) AS SOUTCANTDESP, 
    SUM(ft.SOUTCANTEXIST) AS SOUTCANTEXIST, SUM(ft.SOUTCANTINV) AS SOUTCANTINV, SUM(ft.SOUTMONTODESPBRT) AS SOUTMONTODESPBRT, 
    SUM(ft.SOUTMONTOEXISTBRT) AS SOUTMONTOEXISTBRT, SUM(ft.SOUTMONTOINVBRT) AS SOUTMONTOINVBRT
  FROM PRD_CNS_MX.DM.FACT_SO_SEM_CAD_SKU_INT ft
  LEFT JOIN PRD_STG.GNM_CT.CATSEMANAS cs
    ON ft.SEMID = cs.SEMID
  LEFT JOIN PRD_CNS_MX.CATALOGOS.VW_DIM_PAIS p
    ON p.PAISID = ft.PAISID
  WHERE ft.PAISID IN ({placeholders_pais}) AND cs.SEMANIO*100+cs.SEMNUMERO BETWEEN {semana_inicio-15} AND {semana_fin}
  GROUP BY ft.PAISID, p.PAIS ,ft.SEMID, cs.SEMNUMERO
  ORDER BY ft.PAISID, ft.SEMID;
"""
df_hist = read_sql_upper(Query_hist, conn_sf, PaisSF)

Query_factorbn = f"""
  SELECT 
      f.SEMID,
      f.PAISID,
      f.FBNSELLOUT
  FROM DEV_STG.GNM_CT.FACTORBN AS f
  LEFT JOIN PRD_STG.GNM_CT.CATSEMANAS AS c
      ON f.SEMID = c.SEMID
  WHERE 
      (c.SEMANIO * 100 + c.SEMNUMERO) BETWEEN {semana_inicio} AND {semana_fin}
      AND f.PAISID IN ({placeholders_pais})
      AND f.TIPOCLIENTE = TRUE;
"""

df_fbn = read_sql_upper(Query_factorbn, conn_sf, PaisSF)

El DataFrame resultante está vacío de 
  SELECT 
      f.SEMID,
      f.PAISID,
      f.FBNSELLOUT
  FROM DEV_STG.GNM_CT.FACTORBN AS f
  LEFT JOIN PRD_STG.GNM_CT.CATSEMANAS AS c
      ON f.SEMID = c.SEMID
  WHERE 
      (c.SEMANIO * 100 + c.SEMNUMERO) BETWEEN 202534 AND 202534
      AND f.PAISID IN (%s)
      AND f.TIPOCLIENTE = TRUE;
.


In [48]:
# === Consulta DM ===
query_dm = F"""
 SELECT
    SUM(df.SOUTCANTDESP) SOUTCANTDESP, 
    SUM(df.SOUTMONTODESPBRT) SOUTMONTODESPBRT, 
    c.SEMANIO,
    c.SEMANIO * 100 + c.SEMNUMERO AS SEMNUMERO
FROM  PRD_CNS_MX.DM.FACT_SO_SEM_CAD_SKU_INT AS df
LEFT JOIN PRD_STG.GNM_CT.CATSEMANAS AS c ON df.SEMID = c.SEMID
WHERE c.SEMANIO >= YEAR(CURRENT_DATE) - 1
AND df.PAISID IN ({placeholders_pais})
GROUP BY c.SEMNUMERO, c.SEMANIO ;
"""
query_dm2 = F"""
 SELECT
    df.PAISID,
    c.SEMID,
    SUM(df.SOUTCANTDESP) SOUTCANTDESP, 
    SUM(df.SOUTCANTEXIST) SOUTCANTEXIST,
    SUM(df.SOUTMONTODESPBRT) SOUTMONTODESPBRT, 
    SUM(df.SOUTMONTOEXISTBRT) SOUTMONTOEXISTBRT,
    SUM(df.SOUTMONTOINVBRT) SOUTMONTOINVBRT,
    c.SEMANIO,
    c.SEMANIO * 100 + c.SEMNUMERO AS SEMNUM
FROM  PRD_CNS_MX.DM.FACT_SO_SEM_CAD_SKU_INT AS df
LEFT JOIN PRD_STG.GNM_CT.CATSEMANAS AS c ON df.SEMID = c.SEMID
WHERE SEMNUM BETWEEN {semana_inicio} AND {semana_fin}
AND df.PAISID IN ({placeholders_pais})
GROUP BY c.SEMNUMERO, c.SEMANIO,df.PAISID, c.SEMID;
"""
# === Consulta HEC ===
query_hec = F"""
 SELECT 
    SUM(df.SOUTCANTDESP) SOUTCANTDESP, 
    SUM(df.SOUTMONTODESPBRT) SOUTMONTODESPBRT, 
    c.SEMANIO,
    c.SEMANIO * 100 + c.SEMNUMERO AS SEMNUMERO
FROM  PRD_CNS_MX.SO_HECHOS_INT.FACT_SO_SEM_CAD_SKU_INT AS df
LEFT JOIN PRD_STG.GNM_CT.CATSEMANAS AS c ON df.SEMID = c.SEMID
WHERE c.SEMANIO >= YEAR(CURRENT_DATE) - 1
AND df.PAISID IN ({placeholders_pais})
GROUP BY c.SEMNUMERO, c.SEMANIO ;
"""

In [16]:
df_fbn = read_sql_upper(query_dm2, conn_sf, PaisSF)

El DataFrame resultante está vacío de 
 SELECT
    SUM(df.SOUTCANTDESP) SOUTCANTDESP, 
    SUM(df.SOUTMONTODESPBRT) SOUTMONTODESPBRT, 
    c.SEMANIO,
    c.SEMANIO * 100 + c.SEMNUMERO AS SEMNUMERO
FROM  PRD_CNS_MX.DM.FACT_SO_SEM_CAD_SKU_INT AS df
LEFT JOIN PRD_STG.GNM_CT.CATSEMANAS AS c ON df.SEMID = c.SEMID
WHERE c.SEMNUMERO BETWEEN 202534 AND 202534
AND df.PAISID IN (%s)
GROUP BY c.SEMNUMERO, c.SEMANIO ;
.


### Filtrar archivo

In [17]:
df_CatSem['SEMFIN'] = pd.to_datetime(df_CatSem['SEMFIN'], errors='coerce')
# Ejemplo de DataFrame con intervalos
df_CatSem = df_CatSem.sort_values('SEMINICIO')
if df.empty:
    raise RuntimeError("El DataFrame está vacío después de la carga.")
# DataFrame con fechas a unir
# Asegurar que 'Fecha Ult. Día de la semana' sea datetime
df['Fecha Ult. Día de la semana'] = pd.to_datetime(df['Fecha Ult. Día de la semana'], format='%Y-%m-%d', errors='coerce')
df = df.sort_values('Fecha Ult. Día de la semana')
df = df[df['Fecha Ult. Día de la semana'].notnull()]
# merge_asof une hacia atrás: buscamos el último SEMINICIO <= fecha
df_merge = pd.merge_asof(
    df,
    df_CatSem,
    left_on='Fecha Ult. Día de la semana',
    right_on='SEMINICIO',
    direction='backward'
)
if df_merge.empty or df_merge['SEMFIN'].isna().all():
    raise RuntimeError("No se encontraron fechas dentro del rango de SEMINICIO y SEMFIN.")
# Luego validamos que fecha <= SEMFIN
df_filtrado = df_merge[df_merge['Fecha Ult. Día de la semana'] <= df_merge['SEMFIN']]
df_filtrado = df_filtrado.copy()
df_filtrado['Pais Nombre']= df_filtrado['Pais Nombre'].str.upper().str.strip()
df_filtrado['Nombre Grupo'] = df_filtrado['Nombre Grupo'].str.upper().str.strip()
df_filtrado['Nombre Cadena/Formato'] = df_filtrado['Nombre Cadena/Formato'].str.upper().str.strip()
df_filtrado['EAN'] = df_filtrado['EAN'].astype(str).str.replace(r'\.0$', '', regex=True).str.strip()
# Resumen inicial
resumen = resumir_dataframe(df_filtrado, columnas_sumar, 'df_inicial')

### Rechazos

In [18]:
if df_filtrado.empty:
    raise Exception("El DataFrame está vacío. Revisa los filtros aplicados.")
# Unir los dataframes por las claves correspondientes
df_merged = df_filtrado.merge(df_Pais[['PAIS', 'PAISID']],
                left_on='Pais Nombre',
                right_on='PAIS',
                how='left')

# Hacer el merge para obtener grpid
df_merged2 = df_merged.merge(df_Cliente[['PAISID','GRPNOMBRE', 'GRPID']].drop_duplicates(), #df_Cliente, df_cat_grupos
                            left_on =['PAISID','Nombre Grupo'],
                            right_on = ['PAISID','GRPNOMBRE'],
                            how='left')
rechazo1 = exportar_rechazos_por_campo(df_merged2,'GRPID', ['PAISID',"Nombre Grupo","Nombre Cadena/Formato","SEMID", 'SEMNUMERO'], ["Unidades Desplazadas","Monto Desplazado Bruto ML"], 
                                       carpeta_pais_destino, PaisCarpeta)
# Obtener el cadid
df_merged3 = df_merged2.merge(df_Cliente[['PAISID','CADNOMBRE', 'CADID', 'GRPID']].drop_duplicates(subset =['PAISID','GRPID', 'CADNOMBRE']), #df_Cliente
                            left_on =['PAISID','GRPID','Nombre Cadena/Formato'],
                            right_on = ['PAISID','GRPID', 'CADNOMBRE'],
                            how='left')
df_merged3['GRPID'] = df_merged3['GRPID'].fillna('')
rechazo2 = exportar_rechazos_por_campo(df_merged3,'CADID', ['PAISID','GRPID',"Nombre Grupo","Nombre Cadena/Formato","SEMID", 'SEMNUMERO'], ["Unidades Desplazadas","Monto Desplazado Bruto ML"], 
                            carpeta_pais_destino, PaisCarpeta)
# Convetir al mismo tipo de dato
#df_Prod['PROPSTCODBARRAS'] = df_Prod['PROPSTCODBARRAS'].astype(str).str.replace(r'\.0$', '', regex=True).str.strip()
df_Prod['PROPSTCODBARRAS'] = (df_Prod['PROPSTCODBARRAS'].apply(lambda x: str(int(float(x))) if pd.notnull(x) else None))
# Unir los dataframes por Productos
df_merged4 = df_merged3.merge(df_Prod,
                left_on=['PAISID','EAN'],
                right_on=['PAISID','PROPSTCODBARRAS'],
                how='left')
rechazo3 = exportar_rechazos_por_campo(df_merged4,'PROPSTID', ['PAISID',"EAN","Descripción presentación de producto","SEMID","Cód. Interno ERP","Material SAP", 'SEMNUMERO'], 
                                        ["Unidades Desplazadas","Monto Desplazado Bruto ML"], 
                                        carpeta_pais_destino, PaisCarpeta)
# Resumen del último merge
resumen2 = resumir_dataframe(df_merged4, columnas_sumar,'df_merged')
# Obtener registros que faltan
registros_faltan = df_merged4[df_merged4[["CADID", "GRPID", "PROPSTID"]].isna().any(axis=1)]

Sin rechazos de GRPID
Sin rechazos de CADID
Sin rechazos de PROPSTID


### Agrupar y obtener cálculos

In [19]:
# Montos faltantes
if not registros_faltan.empty:
    # Resumen del último merge
    resumen_faltan = resumir_dataframe(registros_faltan, columnas_sumar,'df_faltan')
    montos_faltan = registros_faltan[['Unidades Desplazadas', 'Monto Desplazado Bruto ML', 'Unidades Inv. Tienda']].sum()
else:
    montos_faltan = pd.Series()

In [20]:
if df_merged4.empty:
    raise Exception("El DataFrame está vacío. Revisa los cruces anteriores.")
# Filtrar filas válidas (sin valores nulos en campos clave)
df_validado = df_merged4.dropna(subset=['CADID', 'GRPID', 'PROPSTID'])
resumen3 = resumir_dataframe(df_validado, columnas_sumar,'df_sin_nulos',montos_faltan)
# Agrupar por claves y sumar los valores numéricos
df_grouped = df_validado.groupby(
    ["SEMID", "PAISID", "CADID", "GRPID", "PROID", "PROPSTID"],
    as_index=False
).agg({
    "Unidades Desplazadas": "sum",
    "Unidades Inv. Tienda": "sum",
    "Unidades Inv. CEDIS": "sum",
    "Unidades Inv. Transito": "sum",
    "Monto Desplazado Bruto ML": "sum",
    "Precio Unitario VTA GLI": "mean",
})
# Renombrar solo las columnas que realmente cambian
df_grouped = df_grouped.rename(columns={
    "Unidades Desplazadas": "SOUTCANTDESP",
    "Unidades Inv. CEDIS": "SOUTCANTCEDIS",
    "Unidades Inv. Transito": "SOUTCANTTRANS",
    "Unidades Inv. Tienda": "SOUTCANTEXIST",
    "Monto Desplazado Bruto ML": "SOUTMONTODESPBRT"
})

if not montos_faltan.empty:
    montos_faltan = montos_faltan.rename({
        "Unidades Desplazadas": "SOUTCANTDESP",
        "Monto Desplazado Bruto ML": "SOUTMONTODESPBRT",
        "Unidades Inv. Tienda": "SOUTCANTEXIST",
    })

In [21]:
if df_grouped.empty:
    raise Exception("El DataFrame está vacío.")
# Calcular inventario total
df_grouped["SOUTCANTINV"] = df_grouped[["SOUTCANTCEDIS", "SOUTCANTTRANS", "SOUTCANTEXIST"]].fillna(0).sum(axis=1)
# Evitar división por cero
cantidades = df_grouped["SOUTCANTDESP"].replace(0, np.nan)
precio_calculado = df_grouped["SOUTMONTODESPBRT"] / cantidades
# Si el cálculo es NaN o 0, usar "Precio Unitario VTA GLI"
df_grouped["PRECIO"] = np.where(
    precio_calculado.isna() | (precio_calculado == 0),
    df_grouped["Precio Unitario VTA GLI"],
    precio_calculado
)
# Calcular montos valorizados
df_grouped["SOUTMONTOEXISTBRT"] = (df_grouped["SOUTCANTEXIST"] * df_grouped["PRECIO"]).fillna(0)
df_grouped["SOUTMONTOINVBRT"] = (df_grouped["SOUTCANTINV"] * df_grouped["PRECIO"]).fillna(0)
# Resúmenes finales
resumen4 = resumir_dataframe(df_grouped, ['SOUTCANTDESP',
    'SOUTCANTEXIST', 'SOUTMONTODESPBRT','SOUTCANTINV',
    'SOUTMONTOEXISTBRT', 'SOUTMONTOINVBRT'],'df_calculado',montos_faltan)
resumen5 = resumir_dataframe(df_grouped, ['SOUTCANTDESP',
    'SOUTCANTEXIST', 'SOUTMONTODESPBRT','SOUTCANTINV',
    'SOUTMONTOEXISTBRT', 'SOUTMONTOINVBRT'], 'df_final')
# Reordenar las columnas
df_final = df_grouped[[
    "PAISID", "SEMID", "PROID", "PROPSTID", "GRPID", 
    "SOUTCANTDESP", "SOUTCANTEXIST", "SOUTCANTINV", 
    "SOUTMONTODESPBRT", "SOUTMONTOEXISTBRT", "SOUTMONTOINVBRT", "CADID"
]]

### Columnas finales con FBN

In [22]:
# Renombrar a columnas brutas
df_final_fbn = df_final.copy() 
fecha_actual = datetime.now().strftime('%Y-%m-%d')
df_final_fbn[['UPDATE_AT', 'CREATE_AT']] = fecha_actual

cols_brt = ['SOUTMONTOEXISTBRT', 'SOUTMONTODESPBRT', 'SOUTMONTOINVBRT']
cols_net = [c.replace('BRT', '') for c in cols_brt]

if not df_fbn.empty:
    # Merge solo con lo necesario y sin duplicados para evitar fan-out
    df_final_fbn = df_final_fbn.merge(df_fbn, on=['SEMID', 'PAISID'], how='left')
    # Factor = 1 + FBN (NaN -> 0)
    factor = 1 + pd.to_numeric(df_final_fbn['FBNSELLOUT'], errors='coerce').fillna(0.0)

    # Calcular netas vectorizado
    net_df = df_final_fbn[cols_brt].mul(factor, axis=0)
    net_df.columns = cols_net
    df_final_fbn[cols_net] = net_df
    # Quitar FBN si ya no se requiere
    df_final_fbn = df_final_fbn.drop(columns=['FBNSELLOUT'])
    
else:
    # Sin FBN: inicializar netas en 0.0 (tu comportamiento original)
    for c in cols_net:
        df_final_fbn[c] = 0.0
resumen6 = resumir_dataframe(df_final_fbn, ['SOUTCANTDESP',
    'SOUTCANTEXIST', 'SOUTMONTODESPBRT','SOUTCANTINV',
    'SOUTMONTOEXISTBRT', 'SOUTMONTOINVBRT', 'SOUTMONTODESP', 'SOUTMONTOEXIST'], 'df_final_fbn')

### Validación final

In [23]:
# Vista de todos los resúmenes
resumen_final = resumen.merge(resumen2, on="Variable", how="outer")
resumen_final = resumen_final.merge(resumen3, on="Variable", how="outer")
resumen_final.replace({"Unidades Desplazadas": "SOUTCANTDESP",
                  "Unidades Inv. Tienda": "SOUTCANTEXIST",
                  "Monto Desplazado Bruto ML": "SOUTMONTODESPBRT"}, inplace=True)
resumen_final = resumen_final.merge(resumen4, on="Variable", how="outer")
resumen_final = resumen_final.merge(resumen5, on="Variable", how="outer")
resumen_final = resumen_final.merge(resumen6, on="Variable", how="outer")
resumen_final.replace(np.nan, '', inplace=True)
resumen_final

df_sum_srzh = df_final.groupby(['PAISID', 'SEMID']).agg({
    'SOUTCANTDESP': 'sum',
    'SOUTCANTEXIST': 'sum',
    'SOUTMONTODESPBRT': 'sum',
    'SOUTMONTOEXISTBRT': 'sum',
    'SOUTMONTOINVBRT': 'sum'
}).reset_index()
df_gp4 = df_merged4.rename(columns={"Unidades Desplazadas": "SOUTCANTDESP",
                  "Unidades Inv. Tienda": "SOUTCANTEXIST",
                  "Monto Desplazado Bruto ML": "SOUTMONTODESPBRT"})

df_sum_crzh = df_gp4.groupby(['PAISID', 'SEMID']).agg({
    'SOUTCANTDESP': 'sum',
    'SOUTCANTEXIST': 'sum',
    'SOUTMONTODESPBRT': 'sum'
}).reset_index()

# Observar gráficas
columnas  = [
    'SOUTCANTDESP',
    'SOUTCANTEXIST',
    'SOUTMONTODESPBRT',
    'SOUTMONTOEXISTBRT',
    'SOUTMONTOINVBRT'
]

# Uso con los tres DataFrames
graficar_por_tipocomprod_interactivo(df_hist, df_sum_crzh, columnas,df_sum_srzh)
# Mostrar rechazos
print('🔹 Rechazos de clientes')
display(rechazo1)
print('\n🔹 Rechazos de cadenas')
display(rechazo2)
print('\n🔹 Rechazos de EAN')
display(rechazo3)
print('\n🔹 Resumen final')
display(resumen_final)

🔹 Rechazos de clientes


None


🔹 Rechazos de cadenas


None


🔹 Rechazos de EAN


None


🔹 Resumen final


,Variable,Resumen_df_inicial,Resumen_df_merged,Resumen_df_sin_nulos,Resumen_df_calculado,Resumen_df_final,Resumen_df_final_fbn
0,Precio Unitario VTA GLI,"104,449.68","104,449.68","104,449.68",,,
1,SOUTCANTDESP,"157,213.22","157,213.22","157,213.22","157,213.22","157,213.22","157,213.22"
2,SOUTCANTEXIST,"765,086.06","765,086.06","765,086.06","765,086.06","765,086.06","765,086.06"
3,SOUTCANTINV,,,,"765,086.06","765,086.06","765,086.06"
4,SOUTMONTODESP,,,,,,0.00
5,SOUTMONTODESPBRT,"2,574,362.00","2,574,362.00","2,574,362.00","2,574,362.00","2,574,362.00","2,574,362.00"
6,SOUTMONTOEXIST,,,,,,0.00
7,SOUTMONTOEXISTBRT,,,,"20,018,152.28","20,018,152.28","20,018,152.28"
8,SOUTMONTOINVBRT,,,,"20,018,152.28","20,018,152.28","20,018,152.28"
9,Total Filas en df,"1,265.00","1,265.00","1,265.00","1,221.00","1,221.00","1,221.00"


In [24]:
raise SystemExit("Stop para revisión")

SystemExit: Stop para revisión

c:\Users\elsilva\Documents\MAY\Carga_Internacional_V2\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



### Insertar en Hechos

In [27]:

if not df_final_fbn.empty:
    semids = sorted({int(x) for x in df_final_fbn['SEMID'].dropna().astype(int)})
    paises = sorted({str(x) for x in PaisSF if x is not None and str(x).strip()})

    delete_sql = """
    DELETE FROM PRD_CNS_MX.SO_HECHOS_INT.FACT_SO_SEM_CAD_SKU_INT t
    WHERE t.SEMID IN (
    SELECT TO_NUMBER(value::string)
    FROM TABLE(FLATTEN(INPUT => PARSE_JSON(%s)))
    )
    AND t.PAISID IN (
    SELECT value::string
    FROM TABLE(FLATTEN(INPUT => PARSE_JSON(%s)))
    );
    """

    cursor = conexion.cursor()
    try:
        conexion.autocommit(False)
        cursor.execute(delete_sql, (json.dumps(semids), json.dumps(paises)))
        print("DELETE ok. Filas afectadas en FACT_SO_SEM_CAD_SKU_INT :", cursor.rowcount)

        success, nchunks, nrows, output = write_pandas(
            conn=conexion,
            df=df_final_fbn,
            database='PRD_CNS_MX',
            schema='SO_HECHOS_INT',
            table_name='FACT_SO_SEM_CAD_SKU_INT'
        )
        if success:
            conexion.commit()
            print(f"Insertadas {nrows} filas.")
        else:
            conexion.rollback()
            print("La carga falló. ROLLBACK.")
    except ProgrammingError as e:
        conexion.rollback()
        print("Error Snowflake:", e)
    finally:
        conexion.autocommit(True)



DELETE ok. Filas afectadas en FACT_SO_SEM_CAD_SKU_INT : 1221
Insertadas 1221 filas.


### Validación datos insertados vs df_final

In [31]:
# Gráfica que compara lo que hay en hechos y en DM
df_dm = read_sql_upper(query_dm, conn_sf, PaisSF)
df_hec = read_sql_upper(query_hec, conn_sf, PaisSF)
generar_grafico_comparativo(
    df_hec, 
    df_dm, 
    campo_metrica="SOUTMONTODESPBRT",
    titulo="Comparativo HEC vs DM con Banda de Desviación",
    archivo_salida="pagina_figura.pdf",
    mostrar_bandas=True,
    
    referencia_estadisticas="prd"
)

In [ ]:
raise SystemExit("Stop para revisión")

In [29]:
# Si todo está bien, hay que llamar al sp para insertar a DM
try:
    paisid = ",".join(sorted({str(x).strip() for x in PaisSF if x}))

    sql_call = f"""
        CALL PRD_STG.GNM_MEX.SP_COPIAR_FACT_SO_SEM_CAD_SKU_DM('{paisid}', '', {semana_inicio}, {semana_fin});
    """

    cursor.execute(sql_call)

    # Si el SP devuelve un resultset (tabla)
    result = cursor.fetchall()
    if result:
        print("Resultado devuelto por el SP:")
        for row in result:
            print(row)

    # Si el SP devuelve un único valor (RETURN simple)
    one = cursor.fetchone()
    if one:
        print("Valor devuelto por el SP:", one[0])

except Exception as e:
    print("Error al ejecutar el SP:", e)


Resultado devuelto por el SP:
('Registros eliminados= 0, copiados= 1221 con parámetros: PAISES=[10], GRPS=[NO APLICADO] SEMID=[1339-1339] (ANIOMES_INI=202534, ANIOMES_FIN=202534) ',)


In [49]:
# validar datos_insertados
df_dm2 = read_sql_upper(query_dm2, conn_sf, PaisSF)
valida = df_sum_srzh.merge(df_dm2, on=['PAISID', 'SEMID'], how='left', suffixes=('', '_insertado'))
# Crear DataFrame de diferencias
diferencias = pd.DataFrame({
    col: ~np.isclose(valida[col], valida[f"{col}_insertado"], atol=1e-2)
    for col in columnas
})
# Filas donde hay al menos una diferencia
filas_con_dif = diferencias.loc[:, diferencias.any()].any(axis=1)
# Si hay diferencias
if filas_con_dif.any():
    # Columnas a mostrar
    columnas_dif = list(diferencias.columns)
    columnas_mostrar = ['PAISID', 'SEMID'] + sum([[col, f"{col}_insertado"] for col in columnas_dif], [])
    # DataFrame con diferencias
    filas_diferentes = valida.loc[filas_con_dif, columnas_mostrar]
    # Estilo: resaltar las diferencias en rojo
    print("Diferencias encontradas:")
    display( filas_diferentes.style.apply(
        lambda row: resaltar_diferencias(row, columnas, diferencias),
        axis=1
    ))
else:
    print("No se encontraron diferencias.")


No se encontraron diferencias.
